### Heroes Of Pymoli Data Analysis
* Female players only make up 14.06% of total players compared to Male players (84.06%), but they out spend male players by $0.40 based on average total purchase per person.

* The highest spending (per person) age demographic falls between 35-39 ($4.76), but they only make up 5.38% of the total players

* Oathbreaker, Nirvana, and Fiery Glass Crusader are listed in both the top five most profitable items and the top five most popular items.
  
-----

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

In [2]:
#Dataframe totaling the number of players who have purchased an item
total_players = len(purchase_data['SN'].unique())
total_players_df = pd.DataFrame({'Total Players': [total_players]})
total_players_df.head()

,Total Players
0,576


## Purchasing Analysis (Total)

In [3]:
#Summary table of the purchasing data
purchasing_summary_df = pd.DataFrame({'Number of Unique Items': [len(purchase_data['Item ID'].unique())],
                                   'Average Price': [purchase_data['Price'].mean()],
                                  'Number of Purchases': [purchase_data['Purchase ID'].count()], 
                                   'Total Revenue': [purchase_data['Price'].sum()]})

#Formatting and displaying the summary table
purchasing_summary_df['Average Price'] = purchasing_summary_df['Average Price'].map("${:.2f}".format)
purchasing_summary_df['Total Revenue'] = purchasing_summary_df['Total Revenue'].map("${:,.2f}".format)
purchasing_summary_df.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

In [4]:
#Create Dataframe of the count of male, female, and undisclosed players using the groupby function
player_gender = purchase_data.drop_duplicates(subset='SN') #removing duplicate SN's because we want that count of players not purchases
gender_demo = player_gender.groupby('Gender')
gender_demo_df = pd.DataFrame(player_gender['Gender'].value_counts())
gender_demo_df = gender_demo_df.rename(columns={'Gender': 'Total Count'})

#Adding a column for the percentage of players for each gender designation
gender_demo_df['Percentage of Players'] = (gender_demo_df['Total Count'] / total_players) * 100
gender_demo_df['Percentage of Players'] = gender_demo_df['Percentage of Players'].map("{:.2f}".format)

#Display the dataframe
gender_demo_df.head()

,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

In [5]:
#Group purchases by Gender
gender_purchase = purchase_data.groupby(['Gender'])

#Create a summary table of purchasing data based on gender
gender_purchase_df = pd.DataFrame({'Purchase Count': gender_purchase['Purchase ID'].count(),
                                  'Average Purchase Price': gender_purchase['Price'].mean(),
                                  'Total Purchase Value': gender_purchase['Price'].sum(),
                                  'Avg Total Purchase per Person': gender_purchase['Price'].sum() / \
                                  gender_demo_df['Total Count']}) #Grab the total count of each gender from the demographics summary table

#Format and display summary table for purchasing data based on gender
gender_purchase_df['Average Purchase Price'] = gender_purchase_df['Average Purchase Price'].map("${:.2f}".format)
gender_purchase_df['Total Purchase Value'] = gender_purchase_df['Total Purchase Value'].map("${:,.2f}".format)
gender_purchase_df['Avg Total Purchase per Person'] = gender_purchase_df['Avg Total Purchase per Person'].map("${:,.2f}".format)
del gender_purchase_df.index.name
gender_purchase_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [6]:
#Create the bins and the labels for them
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
age_ranges = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

#Create a new set of data to add the bins to
age_purchase_data = purchase_data.copy()

#Add bins to the data
age_purchase_data['Age Range'] = pd.cut(age_purchase_data['Age'], bins, labels=age_ranges)

#Create the summary data frame to hold the total count and percentages by age group
age_purchase_data_demo = age_purchase_data.drop_duplicates(subset='SN') #removing duplicate SN's because we want that count of players not purchases
age_demo = age_purchase_data_demo.groupby('Age Range')
age_demo_df = pd.DataFrame(age_demo['SN'].count())
age_demo_df = age_demo_df.rename(columns={'SN': 'Total Count'})
del age_demo_df.index.name
age_demo_df['Percentage of Players'] = (age_demo_df['Total Count'] / total_players) * 100

#Format and display
age_demo_df['Percentage of Players'] = age_demo_df['Percentage of Players'].map("{:.2f}".format)
age_demo_df.head(8)

,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

In [7]:
#Group the binned data by Age Range
purchase_age_analysis = age_purchase_data.groupby(['Age Range'])

#Create a summary table of the data based on Age Range
purchase_age_analysis_df = pd.DataFrame({'Total Count': purchase_age_analysis['Purchase ID'].count(),
                                        'Average Purchase Price': purchase_age_analysis['Price'].mean(),
                                        'Total Purchase Value': purchase_age_analysis['Price'].sum(),
                                        'Avg Total Purchase per Person': purchase_age_analysis['Price'].sum() / \
                                        age_demo_df['Total Count']}) #Grab the count of players from the age demographics table above

#format and display the table
del purchase_age_analysis_df.index.name
purchase_age_analysis_df['Average Purchase Price'] = purchase_age_analysis_df['Average Purchase Price'].map("${:.2f}".format)
purchase_age_analysis_df['Total Purchase Value'] = purchase_age_analysis_df['Total Purchase Value'].map("${:,.2f}".format)
purchase_age_analysis_df['Avg Total Purchase per Person'] = purchase_age_analysis_df['Avg Total Purchase per Person'].map("${:.2f}".format)
purchase_age_analysis_df.head(8)

,Total Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

In [8]:
#Group by SN to pull data by individual spender
spenders = purchase_data.groupby(['SN'])

#DataFrame for the summary of spenders
spenders_df = pd.DataFrame({'Purchase Count': spenders['Purchase ID'].count(),
                           'Average Purchase Price': spenders['Price'].sum() / spenders['Purchase ID'].count(),
                           'Total Purchase Value': spenders['Price'].sum()})

#Sort the dataframe to show the top spenders
top_spenders_df = spenders_df.sort_values('Total Purchase Value', ascending=False)

#Sort and display the summary table
top_spenders_df['Average Purchase Price'] = top_spenders_df['Average Purchase Price'].map("${:.2f}".format)
top_spenders_df['Total Purchase Value'] = top_spenders_df['Total Purchase Value'].map("${:.2f}".format)
top_spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [9]:
#Group raw data by Item ID and Item Name
items = purchase_data.groupby(['Item ID', 'Item Name'])

#Create a dataframe to summarize total purchase count, item price, and total purchase value of each item
items_df = pd.DataFrame({'Purchase Count': items['Item ID'].count(),
                        'Item Price': items['Price'].sum() / items['Item ID'].count(),
                         'Total Purchase Value': items['Price'].sum()})

#Sort to find the most popular items
popular_items_df = items_df.sort_values('Purchase Count', ascending=False)

#Format and display
popular_items_df['Item Price'] = popular_items_df['Item Price'].map("${:.2f}".format)
popular_items_df['Total Purchase Value'] = popular_items_df['Total Purchase Value'].map("${:.2f}".format)
popular_items_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

In [10]:
#Sort items_df to find the most profitable items
profit_items_df = items_df.sort_values('Total Purchase Value', ascending=False)

#Format and display
profit_items_df['Item Price'] = profit_items_df['Item Price'].map("${:.2f}".format)
profit_items_df['Total Purchase Value'] = profit_items_df['Total Purchase Value'].map("${:.2f}".format)
profit_items_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
